# 各種インポート

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q timm

     |████████████████████████████████| 509 kB 7.7 MB/s 


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import timm

# データとモデルの読み込み

In [ ]:
test_images = np.load('/content/drive/MyDrive/Competition/ProbSpace/日本画/data/test_data.npz')['arr_0']
test_images = test_images.transpose((0,3,1,2))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(test_images).float())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, num_workers=2, pin_memory=True)

In [ ]:
model_name = 'tf_efficientnet_b3_ns'
#'tf_efficientnet_b3_ns'
#'seresnext50_32x4d', 'efficientnetv2_rw_s'
num_classes = 4
# 学習済みモデル
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update (2)/fold1_best0728.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update (2)/fold2*0729.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update (2)/fold0_ep40.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update (2)/fold3_best_update.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update (2)/fold4_best_update.pth')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold1*0728.pth')
#weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold2*0729.pth')
#weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold0_best0?.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold0_*.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold2_best0?.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold3_best_update.pth')
#weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold4_best_update.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold4_best0?.pth')

In [ ]:
weights

['/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold1_best0728.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold0_ep40.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold2_best02.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold3_best_update.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold4_best03.pth']

# 推論

In [ ]:
def softmax(a):
    c = np.max(a) # 最大値
    exp_a = np.exp(a - c) # 分子:オーバーフロー対策
    sum_exp_a = np.sum(exp_a) # 分母
    y = exp_a / sum_exp_a 
    return y

In [ ]:
all_predicts = np.zeros((2000, 4))
#all_predicts = np.zeros((len(test_images), 4))
with torch.no_grad():
  for w in weights:
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load(w))
    mode = model.to(device)
    model.eval()
    predicts = []
    for x in test_loader:
      x = x[0].to(device)
      outputs = model(x)
      
      #x = x[0].to(device)
      #outputs = model(x)
      #preds = torch.max(outputs, dim=1)[1]
      tmp = outputs.cpu().numpy()
      #print(outputs.cpu().numpy().shape)
      #print(tmp)
      
      # softmax
      for i, l in enumerate(tmp):
        #print(l)
        #print(i)
        #print(tmp[i])
        tmp[i] = softmax(tmp[i])
        #print(tmp[i])

      predicts.append(tmp)

    all_predicts += np.concatenate(predicts)

In [ ]:
model_name = 'efficientnetv2_rw_s'
#'tf_efficientnet_b3_ns'
#'seresnext50_32x4d', 'efficientnetv2_rw_s'

# 学習済みモデル
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2_true/fold0_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2_true/fold1_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2_true/fold2_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2_true/fold3_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2_true/fold4_best03.pth')

In [ ]:
with torch.no_grad():
  for w in weights:
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load(w))
    mode = model.to(device)
    model.eval()
    predicts = []
    for x in test_loader:
      x = x[0].to(device)
      outputs = model(x)
      
      #x = x[0].to(device)
      #outputs = model(x)
      #preds = torch.max(outputs, dim=1)[1]
      tmp = outputs.cpu().numpy()
      #print(outputs.cpu().numpy().shape)
      #print(tmp)
      
      # softmax
      for i, l in enumerate(tmp):
        #print(l)
        #print(i)
        #print(tmp[i])
        tmp[i] = softmax(tmp[i])
        #print(tmp[i])

      predicts.append(tmp*0.5)

    all_predicts += np.concatenate(predicts)

In [ ]:
model_name = 'seresnext50_32x4d'
#'tf_efficientnet_b3_ns'
#'seresnext50_32x4d', 'efficientnetv2_rw_s'

# 学習済みモデル
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2/fold0_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2/fold1_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2/fold2_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2/fold3_best03.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2/fold4_best03.pth')

In [ ]:
with torch.no_grad():
  for w in weights:
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load(w))
    mode = model.to(device)
    model.eval()
    predicts = []
    for x in test_loader:
      x = x[0].to(device)
      outputs = model(x)
      
      #x = x[0].to(device)
      #outputs = model(x)
      #preds = torch.max(outputs, dim=1)[1]
      tmp = outputs.cpu().numpy()
      #print(outputs.cpu().numpy().shape)
      #print(tmp)
      
      # softmax
      for i, l in enumerate(tmp):
        #print(l)
        #print(i)
        #print(tmp[i])
        tmp[i] = softmax(tmp[i])
        #print(tmp[i])

      predicts.append(tmp*0.6)

    all_predicts += np.concatenate(predicts)

In [ ]:
all_predicts.sum(axis=-1)

array([10.50000022, 10.49999994, 10.50000032, ..., 10.49999994,
       10.49999991, 10.49999998])

In [ ]:
print(all_predicts.shape)
results = np.argmax(all_predicts, axis=1)
results.shape

(2000, 4)


(2000,)

In [ ]:
sub = pd.DataFrame(results, columns=['y'])
sub.index.name = 'id'
sub.index += 1

In [ ]:
sub.to_csv('/content/drive/MyDrive/Competition/ProbSpace/日本画/submission/submission_takasan_ensemble_b3_05v2_06seres_foldall_exmodel.csv')